# MASH posterior contrast

This notenook is designed for different MIGA dataset ... There could be different batches of same cell type. 

In [ ]:
nohup sos run /mnt/mfs/hgrcgrid/homes/rf2872/codes/CUMC_fungun_xqtl_analysis/analysis/Daily_documents/drafts/2023_5/mash_posterior_contrast.ipynb mash_posterior_contrast    \
--orig_file   <(cat /mnt/vast/hpc/csg/rf2872/Work/MASH_test_csg/output/ALL_Ast_End_Exc_Inh_Mic_OPC_Oli.merged_rds.list | cut -f 2) \
-s build -J 200 -q csg2 -l t_pri -c ~/test/csg.yml   &> mash_posterior_contrast.log &




In [ ]:
nohup sos run /mnt/mfs/hgrcgrid/homes/rf2872/codes/CUMC_fungun_xqtl_analysis/analysis/Daily_documents/drafts/2023_5/mash_posterior_contrast.ipynb mash_posterior_contrast    \
--posterior_file   <(cat /mnt/vast/hpc/csg/rf2872/Work/SuSiE_MASH/Secondary_signals/MASH_6_celltypes_promoter_udr_miss/posterior_list | cut -f 2|head) \
--posterior_file_list /mnt/vast/hpc/csg/rf2872/Work/SuSiE_MASH/Secondary_signals/MASH_6_celltypes_promoter_udr_miss/posterior_list \
--orig_file_list /mnt/vast/hpc/csg/rf2872/Work/MASH_test_csg/output/ALL_Ast_End_Exc_Inh_Mic_OPC_Oli.merged_rds.list  \
--cwd MASH_6_celltypes_promoter_udr_miss/contrast \
-s force -J 200 -q csg2 -l t_pri -c ~/test/csg.yml   &> ca.log &


In [ ]:
cd /home/rf2872/Work/Multivariate/MASH/From_SuSiE/2023.5_new/ep_MiGA//
nohup sos run /home/rf2872/codes/CUMC_fungun_xqtl_analysis/analysis/Daily_documents/drafts/2023_6/mash_posterior_merge_contrast.ipynb mash_posterior_merge_contrast    \
--posterior_file   <(cat ~/Work/Multivariate/MASH/From_SuSiE/2023.5_new/ep_MiGA/MASH_6_celltypes_udr_miss/mash_output_list_AD.b_Var | cut -f 2) \
--posterior_file_list ~/Work/Multivariate/MASH/From_SuSiE/2023.5_new/ep_MiGA/MASH_6_celltypes_udr_miss/mash_output_list_AD.b_Var \
--orig_file_list /home/rf2872/2023.5_new/ep_MiGA/output/ROSMAP_Pseudo_eQTL_DLPFC_pQTL_MiGA.merged_rds.list  \
--cwd MASH_6_celltypes_udr_miss/contrast_merge \
--cells "Ast","Exc","Inh","Mic","OPC","Oli","DLPFC_pQTL","MiGA_GFM","MiGA_GTS","MiGA_SVZ","MiGA_THA" \
-s build -J 200 -q csg -c ~/test/csg.yml   &> mash_posterior_contrast.log &


In [ ]:
[global]
import os
# Work directory & output directory
parameter: cwd = path('./')
# The filename prefix for output data
parameter: name="test"
parameter: cells = ["Ast","Exc","Inh","Mic","OPC","Oli","DLPFC_pQTL"]#order is important
parameter: specific_cells = ['Mic','MiGA_GFM','MiGA_GTS','MiGA_SVZ','MiGA_THA']#order is important
parameter: job_size = 1
parameter: container = ''
parameter: table_name = ""
parameter: posterior_file_list = "w"
#parameter: orig_file = path
parameter: posterior_file = path
parameter: orig_file_list = "/mnt/vast/hpc/csg/rf2872/Work/MASH_test_csg/output/ALL_Ast_End_Exc_Inh_Mic_OPC_Oli.merged_rds.list"
#parameter:contrast_dir = "/home/rf2872/Work/MASH_test_csg/MASH_6_celltypes_Dan/contrast/"
##  conditions can be excluded if needs arise. If nothing to exclude keep the default 0
parameter: datadir = ""
import pandas as pd
#parameter: analysis_units = path
# handle N = per_chunk data-set in one job
parameter: per_chunk = 1
regions = [x.replace("\"","").strip().split() for x in open(posterior_file).readlines() if x.strip() and not x.strip().startswith('#')]




In [ ]:
# perform mash posterior contrast for sliced data
[mash_posterior_merge_contrast_1]
input: regions, group_by = 1
output: f"{cwd}/{_input:bnn}_posterior_contrast.rds"
task: trunk_workers = 1, trunk_size = job_size, walltime = '24h',  mem = '10G', tags = f'{_output:bn}'  

R: expand = "${ }",stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout' 
        library(mashr)
        library(RhpcBLASctl)
        blas_set_num_threads(1)
        library(magrittr)
        library(tidyverse)

        #if (!file.exists(contrast_file)) {
        MakePairwiseContrastCols <- function(contrast_left, orig_vector) {
            orig_vector[contrast_left[1]] <- 1
            orig_vector[contrast_left[2]] <- -1
            orig_vector
        }

        FitContrast <- function(index, orig_mean, posterior_mean, posterior_vcov) {
            population_names <- colnames(posterior_mean) %>% str_remove_all("BETA_")

            orig_mean_vector <- orig_mean[index,]
            names(orig_mean_vector) <- population_names
            orig_mean_nonzero <- as.vector(orig_mean_vector != 0)
            orig_mean_tested <- names(orig_mean_vector[orig_mean_nonzero])
            n_populations <- length(orig_mean_tested)

            pairwise_vector <- rep(0, n_populations)
            names(pairwise_vector) <- orig_mean_tested

            grouping<-grouping_all[orig_mean_tested]
            if (n_populations > 1) {
                if (n_populations > 2) {
                    #deviation contrast
                    deviation_contrasts <- rep(-1, n_populations^2) %>% matrix(nrow = n_populations, ncol = n_populations)
                    diag(deviation_contrasts) <- n_populations - 1
                    rownames(deviation_contrasts) <- orig_mean_tested
                    colnames(deviation_contrasts) <- orig_mean_tested
                    deviation_contrasts_tested <- deviation_contrasts[, orig_mean_tested]
                    #microglia populations would get 1/n_Mic for their weight, 
                    diag(deviation_contrasts_tested)[grouping == 2] <- (n_populations - 1) / length(grouping[grouping == 2])
                    deviation_contrasts_tested[grouping == 2, grouping == 2] <- (n_populations - 1) / length(grouping[grouping == 2])
                    colnames(deviation_contrasts_tested) %<>% str_c("_deviation")

                    #pairwise contrast
                    two_combn <- combn(orig_mean_tested, m = 2)
                    pairwise_names <- apply(two_combn, 2, str_c, collapse = "_vs_")
                    pairwise_contrast <- apply(two_combn, 2, MakePairwiseContrastCols, pairwise_vector)
                    colnames(pairwise_contrast) <- pairwise_names
                    #microglia populations would get 1/n_Mic for their weight, 
                    #and Mic vs Mic would still be 1 vs -1 to estimate the internal difference among microglia datasets
                    for (c in 1:ncol(pairwise_contrast)) {
                        if (pairwise_names[c] %>% stringr::str_split(., "_vs_", simplify = T) %>% as.character() %>% grouping[.] %>% sum() < 4) {
                            pairwise_contrast[grouping == 2, c] <- pairwise_contrast[grouping == 2, c] %>% .[which.max(abs(.))] / length(grouping[grouping == 2])
                        }
                    }

                    contrast_design <- cbind(deviation_contrasts_tested / (n_populations - 1), pairwise_contrast)

                } else {
                    pairwise_vector[orig_mean_tested[1]] <- 1
                    pairwise_vector[orig_mean_tested[2]] <- -1
                    contrast_design <- as.matrix(pairwise_vector)
                    colnames(contrast_design) <- str_c(orig_mean_tested[1], "_vs_", orig_mean_tested[2])
                }

                posterior_mean_subset <- posterior_mean[index,]
                names(posterior_mean_subset) %<>% str_remove_all("BETA_")
                posterior_mean_subset2 <- posterior_mean_subset[orig_mean_tested]
                posterior_vcov_subset <- posterior_vcov[,,index]
                colnames(posterior_vcov_subset) %<>% str_remove_all("BETA_")
                rownames(posterior_vcov_subset) %<>% str_remove_all("BETA_")
                posterior_vcov_subset2 <- posterior_vcov_subset[orig_mean_tested,orig_mean_tested]

                contrast_diff <- t(contrast_design) %*% posterior_mean_subset2
                contrast_vcov <- t(contrast_design) %*% posterior_vcov_subset2 %*% contrast_design
                contrast_se <- diag(contrast_vcov) %>% sqrt

                contrast_p <- 2 * (1 - pnorm(abs(contrast_diff) / contrast_se))

                contrast_diff_df <- t(contrast_diff) %>% as_tibble
                colnames(contrast_diff_df) %<>% str_c("mean_contrast_", .)
                contrast_se_df <- t(contrast_se) %>% as_tibble
                colnames(contrast_se_df) %<>% str_c("se_contrast_", .)
                contrast_p_df <- t(contrast_p) %>% as_tibble
                colnames(contrast_p_df) %<>% str_c("p_contrast_", .)

                contrast_df <- bind_cols(contrast_diff_df, contrast_se_df, contrast_p_df)
            } else if(grouping[orig_mean_tested][1]!=grouping[orig_mean_tested][2]){
                contrast_vector <- rep(NA, length(population_names))
                names(contrast_vector) <- str_c("mean_contrast_", population_names, "_deviation")
                contrast_df <- t(contrast_vector) %>% as_tibble
            }
            contrast_df
        }
            cells<-c("${",".join(cells)}")%>%str_split(.,",",simplify = T)%>%as.character 
            specific_cells<-c("${",".join(specific_cells)}")%>%str_split(.,",",simplify = T)%>%as.character 

            grouping_all <- rep(1,length(cells))
            names(grouping_all)<-cells
            grouping_all[specific_cells] <- 2
                
            posterior_file_list<-read.table("${posterior_file_list}")
            orig_file_list<-read.table("${orig_file_list}")
            #get gene name
            gene<-posterior_file_list$V1[posterior_file_list$V2=="${_input}"]
            #find corresponding file path for orig data
            orig_file<- orig_file_list$V2[orig_file_list$V1==gene]
            #read data file
            orig_data <- read_rds(orig_file)$bhat
            posterior_data <- read_rds("${_input}")
            
            posterior_mean <- posterior_data$PosteriorMean
            posterior_cov <- posterior_data$PosteriorCov
            orig_data<-orig_data[,colnames(posterior_mean),drop=FALSE]
            #orig_data[is.na(orig_data)]<-0#just for test
            orig_data[which(is.nan(orig_data))]<-0#just for test
        
            
            
            contrast_result <- map(1:nrow(posterior_mean), FitContrast, orig_data, posterior_mean, posterior_cov) %>% bind_rows %>%
                select(matches("mean_contrast.*deviation"), matches("mean_contrast.*_vs_"), 
                    matches("se_contrast.*deviation"), matches("se_contrast.*_vs_"), 
                    matches("p_contrast.*deviation"), matches("p_contrast.*_vs_"))
            rownames(contrast_result)<-rownames(posterior_mean)
            write_rds(contrast_result,  ${_output:r})
        #}

In [1]:
# merge the contrast data with slice data
[mash_posterior_merge_contrast_2]
input: group_by = "all"
output: f"{cwd}/posterior_sum.csv"
task: trunk_workers = 1, trunk_size = job_size, walltime = '24h',  mem = '10G', tags = f'{_output:bn}'  

R: expand = "${ }",stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout' 
        library(dplyr)
        library(tidyverse)
        library(ggnewscale)
        
        all.list <- stringr::str_split("${_input}", " ", simplify = T)
       
        p_cut <- 1E-05

        cells<-c("${",".join(cells)}")%>%str_split(.,",",simplify = T)%>%as.character #ggnewscale cannot use a specified order, I can not find a good way to order them by category for now
        conditions <-combn(cells, m = 2) %>%apply(., 2, str_c, collapse = "_vs_") 
        

        df <- matrix(ncol = length(conditions), nrow = 4) %>% as.data.frame()
        colnames(df) <- conditions
        rownames(df) <- c( "n_sig_snp", "n_snp","n_sig_feature","n_all_feature")
        for (con in conditions) {
            print(con)
            n.all.sig.snp <- 0
            n.all.snp <- 0
            n.all.sig.feature <- 0
            n.all.feature <- 0

            for (i in 1:length(all.list)) {
                print(i)
                tmp <- readRDS(all.list[i])
                p.mtx <- tmp %>% select(matches("p_contrast.*_vs_"))
                p.mtx.con <- p.mtx %>% select(matches(con))
                n.sig.snp <- sum(p.mtx.con < p_cut)
                # print(n.sig.snp)
                if(ncol(p.mtx.con)>0){
                    p.mtx.con<-na.omit(p.mtx.con)
                    n.sig.snp <- sum(p.mtx.con < p_cut)
                    n.snp <- nrow(p.mtx.con)
                } else {
                    n.sig.snp <- n.snp <-0
                }
                # print(n.snp)
                n.sig.feature <- ifelse(n.sig.snp > 0, 1, 0)
                n.feature<- ifelse (n.snp > 0 , 1, 0)

                n.all.sig.snp <- n.sig.snp + n.all.sig.snp
                n.all.snp <- n.all.snp + n.snp
                n.all.sig.feature <- n.all.sig.feature + n.sig.feature
                n.all.feature <- n.all.feature + n.feature
            }
            df[, con] <- c( n.all.sig.snp,n.all.snp, n.all.sig.feature,n.all.feature)
        }
        #df["n_all_feature",] <- length(all.list)
        write.csv(df, "${_output}")

In [1]:
# plot contrast result
[mash_posterior_merge_contrast_3]
input: group_by = "all"
output: f"{cwd}/posterior_sum.png"
task: trunk_workers = 1, trunk_size = job_size, walltime = '24h',  mem = '10G', tags = f'{_output:bn}'  

R: expand = "${ }",stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout' 
        library(dplyr)
        library(tidyverse)
        library(ggnewscale)
        df <- read.csv("${_input:a}",row.names=1)
        colnames(df)<-gsub("DLPFC_","",colnames(df))
        for (i in 1:ncol(df)) {
            con1 <- stringr::str_split(colnames(df)[i], "_vs_", simplify = T)[, 1]
            con2 <- stringr::str_split(colnames(df)[i], "_vs_", simplify = T)[, 2]

            if (con1 > con2) {
                new.name <- paste0(con2, "_vs_", con1)
                colnames(df)[i] <- new.name
            }
        }

        ## summarizxse with approach 1: snp-feature pair
        snp.ratio <- df["n_sig_snp", ] / df["n_snp", ]
        snp.ratio <- snp.ratio %>%
            t() %>%
            as.data.frame()
        colnames(snp.ratio) <- "ratio"
        snp.ratio$group <- "snp"

        ## summarize with approach 2: feature
        fet.ratio <- df["n_sig_feature", ] / df["n_all_feature", ]
        fet.ratio <- fet.ratio %>%
            t() %>%
            as.data.frame()
        rownames(fet.ratio) <- paste0(stringr::str_split(rownames(fet.ratio), "_vs_", simplify = T)[, 2], "_vs_", stringr::str_split(rownames(fet.ratio), "_vs_", simplify = T)[, 1])
        colnames(fet.ratio) <- "ratio"
        fet.ratio$group <- "feature"
        ratio <- rbind(snp.ratio, fet.ratio)

        ## I need to add the below to make it Simmetrie

        cons <- rownames(ratio) %>%
            str_split(., "_vs_", simplify = T) %>%
            .[, 1] %>%
            unique()
        for (i in 1:length(cons)) {
            new.name <- paste0(cons[i], "_vs_", cons[i])
            ratio[new.name, ] <- 0
        }

        ratio$con1 <- stringr::str_split(rownames(ratio), "_vs_", simplify = T)[, 1]
        ratio$con2 <- stringr::str_split(rownames(ratio), "_vs_", simplify = T)[, 2]

        ## prepare for the plot, score1 is for snp-feature pair, score2 is for feature only
        ratio$score1 <- ratio$score2 <- 0
        ratio$score1[ratio$group == "snp"] <- ratio$ratio[ratio$group == "snp"]
        ratio$score2[ratio$group == "feature"] <- ratio$ratio[ratio$group == "feature"]
        ratio$label <- paste0(round(ratio$ratio, 4) * 100, "%")
        ratio$label[ratio$group == 0] <- NA

        # plot
        num_cols <- length(cons)
        height <- width <- 4 + num_cols * 0.5 
        ggplot(ratio[ratio$group == "snp", ], aes(x = con1, y = con2)) +
            geom_tile(aes(fill = score1)) +
            scale_fill_gradient2("SNP_Feature pair",
                low = "#762A83", mid = "white", high = "#1B7837"
            ) +
            new_scale("fill") +
            geom_tile(aes(fill = score2), data = subset(ratio, group != "snp")) +
            scale_fill_gradient2("Feature",
                low = "#1B7837", mid = "white", high = "#762A83"
            ) +
            geom_text(data = ratio, aes(label = label)) +
            theme_bw()
        #geom_text(data=ratio, aes(label = label, color = factor(group))) +theme_bw()
        #ggsave(gsub(".csv",".png",filename))

        ggsave("${_output}",width = width, height = height)